<div style='background-color:#193366;color:white;padding:20px;text-align:center;font-size:20px'> Web Scraping model for 1 publication on Medium </div>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
from datetime import datetime

In [2]:
import numpy as np

In [4]:
#page/archive/year/month/day

## Put url of publication

In [5]:
url='https://towardsdatascience.com/archive/{0}/{1:02d}/{2:02d}'

## define function to convert day of year to datetime (apply to 2020)

In [6]:
def convert_day2(day_num): 
    str(day_num).rjust(3 + len(str(day_num)), '0') 
    year = "2020"
    res = datetime.strptime(year + "-" + str(day_num), "%Y-%j")
    return res


In [7]:
convert_day2(32)

datetime.datetime(2020, 2, 1, 0, 0)

## Specify time here

In [8]:
selected_days = [i for i in range(336,367)]

## Scraping

In [9]:
data=[]
for d in selected_days:
    da=convert_day2(d)
    req = requests.get(url.format(da.year, da.month, da.day), allow_redirects=True)
    if not req.url.startswith(url.format(da.year, da.month, da.day)):
        continue
    
    soup=BeautifulSoup(req.content,'lxml')
    #author name and date
    x1=soup.find_all('div',class_="postMetaInline postMetaInline-authorLockup ui-captionStrong u-flex1 u-noWrapWithEllipsis")
    author=[]
    date=[]
    for i1 in x1:
        author.append(i1.find('a').string)
        date.append(da)
    #title
    x3=soup.find_all('div',class_="section-inner sectionLayout--insetColumn")
    title=[]
    for i3 in x3:
        if i3.find('h3') is not None:
            title.append(i3.find('h3').string)
        else:
            title.append(np.nan)
    #min read
    x2=soup.find_all('div',class_="ui-caption u-fontSize12 u-baseColor--textNormal u-textColorNormal js-postMetaInlineSupplemental")
    read_time=[]
    for i2 in x2:
        read_time.append(int(i2.find('span',class_='readingTime')['title'][:-9]))
    #number of clap
    x4=soup.find_all('span',class_="u-relative u-background js-actionMultirecommendCount u-marginLeft5")
    clap=[]
    for i4 in x4:
        if i4.find('button') is None:
            clap.append(np.nan)
        else:
            clap.append(i4.find('button').string)
            
    data.append(pd.DataFrame({'Date':date,'Author':author,'Title':title,'Read time':read_time,'No of Clap':clap}))



## Transform dataframe

In [10]:
DATA=pd.concat(data)

## Save dataset file to local computer

In [11]:
DATA.to_csv('dataTDS_122020.csv',index=False)